In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

class DFA:
    def __init__(self, states, alphabet, transition_function, start_state, accept_states):
        self.states = states
        self.alphabet = alphabet
        self.transition_function = transition_function
        self.start_state = start_state
        self.accept_states = accept_states

    def remove_unreachable_states(self):
        reachable = set()  # Ulaşılabilir durumlar kümesi
        to_process = [self.start_state]
        while to_process:
            current_state = to_process.pop()
            if current_state not in reachable:
                reachable.add(current_state)
                # Geçiş fonksiyonu ile durumlara git
                for symbol in self.alphabet:
                    next_state = self.transition_function.get((current_state, symbol))
                    if next_state and next_state not in reachable:
                        to_process.append(next_state)
        # Ulaşılabilir olmayan durumları kaldır
        self.states = self.states.intersection(reachable)
        self.accept_states = self.accept_states.intersection(reachable)
        self.transition_function = {key: value for key, value in self.transition_function.items() if key[0] in reachable}

    def group_states_by_transition(self):
        groups = {}  # Durum kümeleri
        state_mapping = {}  # Durumların hangi kümeye ait olduğunu gösterir

        # Durumları geçişlerine göre gruplama işlemi
        for state in self.states:
            transition_signature = tuple(self.transition_function.get((state, symbol), None) for symbol in self.alphabet)
            
            # Aynı geçiş fonksiyonuna sahip olan durumları grupla
            if transition_signature not in groups:
                groups[transition_signature] = set()
            groups[transition_signature].add(state)
            state_mapping[state] = transition_signature

        # Grupları döndür
        return groups, state_mapping

    def minimize(self):
        self.remove_unreachable_states()
        groups, state_mapping = self.group_states_by_transition()

        # Grupları güncelle
        self.groups = groups
        self.state_mapping = state_mapping

    def plot_dfa(self):
        G = nx.DiGraph()
        # Durumları ve geçişleri grafiğe ekliyoruz
        for (state, symbol), next_state in self.transition_function.items():
            G.add_edge(state, next_state, label=symbol)

        pos = nx.spring_layout(G)
        plt.figure(figsize=(8, 6))
        nx.draw(G, pos, with_labels=True, node_size=3000, node_color='lightblue', font_size=16, font_weight='bold', arrowsize=20)
        labels = nx.get_edge_attributes(G, 'label')
        nx.draw_networkx_edge_labels(G, pos, edge_labels=labels)

        # Başlangıç durumu işaretleniyor
        plt.text(pos[self.start_state][0], pos[self.start_state][1] + 0.1, 'Start', horizontalalignment='center', fontsize=12, color='green')
        # Kabul durumu işaretleniyor
        for state in self.accept_states:
            plt.text(pos[state][0], pos[state][1] - 0.1, 'Accept', horizontalalignment='center', fontsize=12, color='red')

        plt.title("Başlangıç DFA Durumu")
        plt.show()


# Basitleştirilmiş DFA Tanımlaması
states = {'q0', 'q1', 'q2', 'q3', 'q4', 'q5', 'q6'}
alphabet = {'a', 'b'}
transition_function = {
    ('q0', 'a'): 'q1', ('q0', 'b'): 'q2',
    ('q1', 'a'): 'q3', ('q1', 'b'): 'q4',
    ('q2', 'a'): 'q5', ('q2', 'b'): 'q6',
    ('q3', 'a'): 'q6', ('q3', 'b'): 'q0',
    ('q4', 'a'): 'q6', ('q4', 'b'): 'q1',
    ('q5', 'a'): 'q6', ('q5', 'b'): 'q2',
    ('q6', 'a'): 'q6', ('q6', 'b'): 'q0',
}
start_state = 'q0'
accept_states = {'q6'}  # Kabul durumu q6

# DFA nesnesi oluşturuluyor
dfa = DFA(states, alphabet, transition_function, start_state, accept_states)

# Başlangıç DFA'sının çizimi
dfa.plot_dfa()

# Durum indirgeme işlemi
dfa.minimize()

# Geçiş fonksiyonu gruplamalarıyla birlikte yazdırma
print("İndirgenmiş DFA Grupları:")
for group, states in dfa.groups.items():
    print(f"Grup: {states}, Geçiş Fonksiyonu: {group}")

# Geçiş fonksiyonunu da gruplar şeklinde yazdıralım
print("\nGeçiş Fonksiyonu:")
for group, states in dfa.groups.items():
    # Geçişleri yazdırıyoruz
    for state in states:
        # Geçiş fonksiyonu için her durumu kontrol ediyoruz
        next_state_a = dfa.transition_function.get((state, 'a'))
        next_state_b = dfa.transition_function.get((state, 'b'))
        
        # Geçiş fonksiyonunu yazdırıyoruz
        if next_state_a:
            print(f"{states} -> a -> {next_state_a}")
        if next_state_b:
            print(f"{states} -> b -> {next_state_b}")
